# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
experiment='cxilu9218'
runNumber = 91

# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
print os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic'
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *
# Determine current user
currentUser, error = unixCMD("echo $USER")
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab detector data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

## Load point data

In [ ]:
# pointData = load_obj( os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )
pointData = load_obj('/cds/data/psdm/cxi/cxilu9218/results/mrware/pointData-run-%d' % runNumber)

def dropEmpty(pointData):

    filledBin = (pointData['seconds'] > 0)    

    for key in pointData.keys():
        pointData[key]=pointData[key][filledBin] 
        
    return pointData

def dropNaN(pointData):
    goodBin = np.ones(pointData['seconds'].shape, dtype=bool)
    for key in pointData.keys():
        if not pointData[key].dtype==np.dtype(object) and pointData[key].shape==pointData['seconds'].shape:
            goodBin = goodBin & ~np.isnan(pointData[key])
    for key in pointData.keys():
        pointData[key]=pointData[key][goodBin]
    return pointData
print len(pointData['seconds'])
pointData_masked = dropNaN(dropEmpty(pointData))
print 'Number of good shots: ', len(pointData_masked['seconds'])

## Plot histogram of result

In [ ]:
detArrays = pointData
print detArrays.keys()

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['uvint1'], bins=100,normed=False, alpha=0.5);

# Read in CSPAD via batch job

## Specify good idxs

In [ ]:
goodIdx = runFilter( pointData , filterOn=['ttfltpos','uvint','uvint1'], maddevs=3 ).astype(bool)
gooseIdx = ~pointData['laserOn']
goodIdx = goodIdx & pointData['laserOn'] & pointData['xrayOn']
print 'Number of good shots: ', len(goodIdx[goodIdx==True])
print 'Number of goose shots: ', len(gooseIdx[gooseIdx==True])

## Now generate time delays

In [ ]:
# Calibrated timetool for LU92
TTPoly = np.array([2.95684259e-06, -1.43969413e-03]) 

TTFltPos = pointData['ttfltpos']
TTCorrFltPos = TTPoly[0]*TTFltPos+TTPoly[1]
TTTime = 1e3*TTCorrFltPos - 1e9/3e8*2*(pointData['stageencoder']-56.35)

# NOTE: This is a placeholder, and should be removed on runs with timetool data
# TTTime = np.zeros(pointData['xrayEnergy'].shape)

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(TTTime[goodIdx], bins=100,normed=False, alpha=1)
ax2.set_xlabel('Delay (ps)')
ax2.set_ylabel('Counts')
plt.show()

# Read in all images
## If you just want to look at all the images at once

In [ ]:
# tagList = []
# size_slice = int(len(goodIdx[goodIdx==True])/20)
# print size_slice
# for idx in range(20):
#     filteredSeconds = pointData['seconds'][goodIdx][0+size_slice*idx:size_slice+size_slice*idx]
#     filteredNanoseconds = pointData['nanoseconds'][goodIdx][0+size_slice*idx:size_slice+size_slice*idx]
#     filteredFiducials = pointData['fiducials'][goodIdx][0+size_slice*idx:size_slice+size_slice*idx]
#     tagList.append( {'seconds':filteredSeconds, 'nanoseconds':filteredNanoseconds, 'fiducials':filteredFiducials} )

# # batchThreads = batchCSPADGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')
# batchThreads = batchCSPADMVGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')
# timebins=np.arange(10)
# batchThreads.Queue = 'psanaq'
# batchThreads.RunType = 'mpirun python2'
# batchThreads.start()

In [ ]:
print np.unique(TTTime)
print np.unique(pointData['stageencoder'])

# Read in timebinned CSPAD
## If you want to timebin your images

In [ ]:
# timebins = np.arange( -1.5,1.5,.03 )
# timebins are in picoseconds
# timebins = np.arange( 2.,5.5,.03 ) 
# timebins = np.arange( -2, 2, .05 )
# timebins = np.array([-1.25,0.75])
# timebins = np.unique(TTTime)
def rebin_stage_positions(stage_positions):
    unique = np.sort(np.unique(stage_positions))
    unique_binned = []
    for i, val in enumerate(unique[:-1]):
        if unique[i+1]-unique[i]>0.004:
            unique_binned.append(unique[i])
    unique_binned.append(unique[-1])
    return np.array(unique_binned)
timebins = -1e9/3e8*2*(rebin_stage_positions(pointData['stageencoder'])-56.35)
print 'Number of timebins: ', len(timebins)
# dt = timebins[1]-timebins[0]
dt = np.min(np.diff(timebins))
print 'dt (ps): ',dt

# first timebin is for goose trigger events
# set to very large negative number (laser definitely off)
timebins = np.concatenate((np.array([-100]),timebins))
NT = timebins.size

# print np.around(timebins,3)
# print np.sort(1e9/3e8*2*(rebin_stage_positions(pointData['stageencoder'])-56.35))

tagList = []
for idx, t in enumerate(timebins):
    if idx>0 and idx<len(timebins)-1:
        dt_plus = (timebins[idx+1]-timebins[idx])/2
        dt_minus = (timebins[idx]-timebins[idx-1])/2
    elif idx==0:
        dt_plus = (timebins[idx+1]-timebins[idx])/2
        dt_minus = 1.0
    elif idx==len(timebins)-1:
        dt_plus = 1.0
        dt_minus = (timebins[idx]-timebins[idx-1])/2
    tIndex = (TTTime >= t-dt_minus)&(TTTime < t+dt_plus)
    allIndex = tIndex & goodIdx
    filteredSeconds = pointData['seconds'][allIndex]
    filteredNanoseconds = pointData['nanoseconds'][allIndex]
    filteredFiducials = pointData['fiducials'][allIndex]
    tagList.append( {'seconds':filteredSeconds, 'nanoseconds':filteredNanoseconds, 'fiducials':filteredFiducials} )

# # batchThreads = batchCSPADGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')
# batchThreads = batchCSPADMVGrabber (tagList, experiment=experiment, runNumber=runNumber, detType='Jungfrau')

# batchThreads.Queue = 'psanaq'
# batchThreads.RunType = 'mpirun python2'
# batchThreads.start()


## Check job status

In [ ]:
print batchThreads.status
stdoutdata, stderrdata = bjobs()
print stdoutdata
print stderrdata

## Kill threads if they die

In [ ]:
batchThreads.requestStop()
time.sleep(5)
stdoutdata, stderrdata = bkill(killAll=True)
print stdoutdata
print stderrdata

# Save resulting CSPAD

In [ ]:
batchThreads.gather()
CSPAD = batchThreads.CSPAD[:,:,:,1:]
variance = batchThreads.variance[:,:,:,1:]
counts = batchThreads.counts[1:]
timebins_goose = timebins[0:1]
CSPAD_goose = batchThreads.CSPAD[:,:,:,0]
variance_goose = batchThreads.variance[:,:,:,0]
counts_goose = batchThreads.counts[0:1]

In [ ]:
from scipy.io import savemat
import h5py

# save as numpy array
np.save(os.environ['RESULTSPATH']+'/timebins-run-%d' % runNumber ,timebins)
np.save(os.environ['RESULTSPATH']+'/CSPAD-run-%d' % runNumber  , CSPAD)
np.save(os.environ['RESULTSPATH']+'/variance-run-%d' %  runNumber , variance)
np.save(os.environ['RESULTSPATH']+'/counts-run-%d' % runNumber  , counts)
np.save(os.environ['RESULTSPATH']+'/timebins_goose-run-%d' % runNumber ,timebins_goose)
np.save(os.environ['RESULTSPATH']+'/CSPAD_goose-run-%d' % runNumber  , CSPAD_goose)
np.save(os.environ['RESULTSPATH']+'/variance_goose-run-%d' %  runNumber , variance_goose)
np.save(os.environ['RESULTSPATH']+'/counts_goose-run-%d' % runNumber  , counts_goose)

# save as .mat
# filename = os.environ['RESULTSPATH']+'/timebinned-detector-images-run-%d' % runNumber
# mdict = {'timebins':timebins,'CSPAD':CSPAD,'variance':variance,'counts':counts}
# savemat(filename, mdict)

# save as .h5
filename = os.environ['RESULTSPATH']+'/timebinned-detector-images-run-%d.h5' % runNumber
with h5py.File(filename, 'w') as hf:
    hf.create_dataset("timebins",  data=timebins)
    hf.create_dataset("CSPAD",  data=CSPAD)
    hf.create_dataset("variance",  data=variance)
    hf.create_dataset("counts",  data=counts)
    hf.create_dataset("timebins_goose",  data=timebins_goose)
    hf.create_dataset("CSPAD_goose",  data=CSPAD_goose)
    hf.create_dataset("variance_goose",  data=variance_goose)
    hf.create_dataset("counts_goose",  data=counts_goose)

# Plot CSPAD

In [ ]:
# # Optional code if timebinned frames are already saved
timebins = np.load(os.environ['RESULTSPATH']+'/timebins-run-%d.npy' % runNumber )
CSPAD = np.load(os.environ['RESULTSPATH']+'/CSPAD-run-%d.npy' % runNumber)
variance = np.load(os.environ['RESULTSPATH']+'/variance-run-%d.npy' % runNumber)
counts = np.load(os.environ['RESULTSPATH']+'/counts-run-%d.npy' % runNumber)
timebins_goose = np.load(os.environ['RESULTSPATH']+'/timebins_goose-run-%d.npy' % runNumber )
CSPAD_goose = np.load(os.environ['RESULTSPATH']+'/CSPAD_goose-run-%d.npy' % runNumber)
variance_goose = np.load(os.environ['RESULTSPATH']+'/variance_goose-run-%d.npy' % runNumber)
counts_goose = np.load(os.environ['RESULTSPATH']+'/counts_goose-run-%d.npy' % runNumber)

In [ ]:
for i in range(CSPAD.shape[-1]):
    m = np.mean(CSPAD[:,:,:,i])
    if not np.isnan(m):
        print i, np.mean(CSPAD[:,:,:,i]), counts[i][0], timebins[i]

In [ ]:
from IPython.display import clear_output
from plotStyles import *

def plotCSPAD( cspad , x , y, cspadMask=None, zLims = None, divergent=False, NTILE=8 ):
    figOpts = {'xLims':[-1e5,1e5],'yLims':[-1e5,1e5],'divergent':divergent, 'xIn':3, 'yIn':3*11.5/14.5}
    
    if zLims is not None:
        figOpts['zLims'] = zLims
    
    for iTile in range(NTILE):
    
        if cspadMask is not None:
            cspadTile = cspad[iTile,:,:]
            tileMask = ~cspadMask[iTile,:,:]
            cspadTile[tileMask] = 0
        
        if iTile == 0:
            newFigure = True
        else:
            newFigure = False
            
        clear_output()
        colorPlot( x[iTile,:,:], y[iTile,:,:], cspadTile , newFigure=newFigure, **figOpts);


x,y = CSPADgeometry(detType='Jungfrau', run=runNumber, experiment=experiment)

# cspadMask = createMask(experiment=experiment, run=runNumber, detType='Jungfrau').astype(bool)
cspadMask = np.ones_like(x).astype(bool)
print(cspadMask.shape)

print CSPAD.shape
CSPADbinned = CSPAD


In [ ]:
CSPADbinned = np.copy(CSPAD)
# CSPADbinned[CSPADbinned>2000]=0


from scipy.ndimage.filters import gaussian_filter
sigma = 10

CSPADblurred = np.zeros(CSPAD.shape)
CSPADbinned[CSPADbinned>50]=0
for i in range(CSPAD.shape[-1]):
    if not np.isnan(counts[i][0]):
        for j, tile in enumerate(CSPADbinned[:,:,:,i]):
            CSPADblurred[j,:,:,i] = gaussian_filter(tile, sigma=sigma)


In [ ]:
CSPADsummed = np.nanmean(CSPADbinned, axis=-1)
plotCSPAD( CSPADsummed, x , y , zLims=[0,1.5],
          cspadMask=cspadMask, divergent=False, NTILE=8)
# plt.savefig('sf6_run12.png')

In [ ]:
frame = CSPADbinned[:,:,:,12]
print np.mean(frame), np.std(frame)
print len(frame[frame>0])*0.1/len(frame.flatten())

In [ ]:
print '{0:<5} {1:>5} {2:>8} {3:>8} {4:>8}'.format('Index','Count','Sum','Median','Variance')
for i in range(CSPAD.shape[-1]):
    if counts[i][0]>0:
        vals = (i, counts[i][0], round(np.sum(CSPAD[:,:,:,i])),round(np.median(CSPAD[:,:,:,i]),4), round(np.var(CSPAD[:,:,:,i]),4))
        print '{0:<5} {1:>5} {2:>8} {3:>8} {4:>8}'.format(*vals)

In [ ]:
# x,y = CSPADgeometry(detType='Jungfrau', run=runNumber, experiment=experiment)

# filename = os.environ['RESULTSPATH']+'/detector-geometry'
# mdict = {'x':x,'y':y}
# savemat(filename, mdict)

In [ ]:
plt.figure(figsize=(12,9))
fig2, ax2 = plt.subplots(1, 1)
ax2.hist(TTTime[goodIdx], bins=100,normed=False, alpha=1)
ax2.set_xlabel('Delay (ps)')
ax2.set_ylabel('Counts')
for i,ibin in enumerate(timebins[1:-1]):
    plt.axvline(x=timebins[i+1]+(timebins[i+2]-timebins[i+1])/2., color='r')
plt.show()